In [1]:
%config Completer.use_jedi = False

# Imports

In [1]:
from collections import deque, Counter, OrderedDict
import csv
import empath
import lime
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import numpy as np
import os
import pandas as pd
# from pandas.io.json import json_normalize
import pickle
import re

import seaborn as sns
import string


# import gensim
# from gensim.models.wrappers import LdaMallet
# import spacy

In [2]:
# import lazypredict
from lazypredict.Supervised import LazyClassifier


from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import Isomap, LocallyLinearEmbedding, TSNE
from sklearn.metrics import accuracy_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, Normalizer
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier



# Data

In [16]:
train_data = pd.read_csv(r"data\c50.csv")
test_data = pd.read_csv(r"data\c50_Test.csv")
test_data = test_data.drop(test_data.columns[0], axis=1)

In [17]:
test_data
train_data

,Author,Text
0,candidate00001,The Internet may be overflowing with new techn...
1,candidate00001,The U.S. Postal Service announced Wednesday a ...
2,candidate00001,Elementary school students with access to the ...
3,candidate00001,An influential Internet organisation has backe...
4,candidate00001,An influential Internet organisation has backe...
...,...,...
2495,candidate00050,China's central bank chief has said that infla...
2496,candidate00050,"China ushered in 1997, a year it has hailed as..."
2497,candidate00050,China issued tough new rules on the handling o...
2498,candidate00050,China will avoid bold moves in tackling its ai...


In [18]:
cols = list(test_data.columns)
cols = [cols[-1]] + cols[:-1]
test_data = test_data.reindex(columns=cols)
test_data = test_data.rename(columns={'true-author': 'Author'})
test_data = test_data.drop(columns='ID', axis=1)
test_data

,Author,Text
0,candidate00046,China and Britain agreed on Wednesday to relea...
1,candidate00001,The Federal Reserve may not be taking adequate...
2,candidate00009,Britain's motor industry reported 1996 car reg...
3,candidate00019,When the former Czechoslovak diplomat Josef Ko...
4,candidate00012,China is building a network of major toll high...
...,...,...
2495,candidate00018,Britain's big banks look set to raise profits ...
2496,candidate00047,After two years of hype and euphoria about the...
2497,candidate00002,Czech annual average consumer inflation eased ...
2498,candidate00037,"Kellogg Co, whose profits for 1996 are under p..."


In [19]:
full_data = pd.concat([train_data, test_data])
full_data

,Author,Text
0,candidate00001,The Internet may be overflowing with new techn...
1,candidate00001,The U.S. Postal Service announced Wednesday a ...
2,candidate00001,Elementary school students with access to the ...
3,candidate00001,An influential Internet organisation has backe...
4,candidate00001,An influential Internet organisation has backe...
...,...,...
2495,candidate00018,Britain's big banks look set to raise profits ...
2496,candidate00047,After two years of hype and euphoria about the...
2497,candidate00002,Czech annual average consumer inflation eased ...
2498,candidate00037,"Kellogg Co, whose profits for 1996 are under p..."


## Preprocessing

In [20]:
# remove leading and trailing whitespace
full_data['Text'] = full_data['Text'].str.strip()

# replace multiple spaces with a single space
full_data['Text'] = full_data['Text'].str.replace(r'\s+', ' ')

remove_emails = lambda x: re.sub(r'\S+@\S+', '', x)
remove_urls = lambda x: re.sub(r'http\S+', '', x)

full_data['Text'] = full_data['Text'].apply(remove_emails).apply(remove_urls)
full_data['Text'] = full_data['Text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower()))
full_data['Text'] = full_data['Text'].str.replace('\n', '')

In [21]:
word_count_func = lambda x: len(x.split())
full_data['word_count'] = full_data['Text'].apply(word_count_func)
full_data = full_data.reset_index(drop=True)
full_data

,Author,Text,word_count
0,candidate00001,the internet may be overflowing with new techn...,18
1,candidate00001,the us postal service announced wednesday a pl...,25
2,candidate00001,elementary school students with access to the ...,30
3,candidate00001,an influential internet organisation has backe...,23
4,candidate00001,an influential internet organisation has backe...,23
...,...,...,...
4995,candidate00018,britains big banks look set to raise profits b...,37
4996,candidate00047,after two years of hype and euphoria about the...,31
4997,candidate00002,czech annual average consumer inflation eased ...,19
4998,candidate00037,kellogg co whose profits for 1996 are under pr...,37


In [22]:
categories = pd.Categorical(full_data['Author']).categories
full_data['encoded_Author'] = pd.Categorical(full_data['Author'], categories=categories).codes

full_texts = full_data['Text'].to_list()
full_labels = full_data['encoded_Author'].to_list()


In [23]:
X = full_data['Text']#.drop(['Author', 'encoded_author'], axis=1)
y = full_data['encoded_Author']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
len(X_train), len(X_test), len(X_val), len(y_train), len(y_test), len(y_val)

(3000, 1000, 1000, 3000, 1000, 1000)

In [24]:
train_text = list(train_data['Text'])
train_auth = list(train_data['Author'])

test_text = list(test_data['Text'])
test_auth = list(test_data['Author'])

full_text = list(full_data['Text'])
full_auth = list(full_data['Author'])

## TFIDF 

In [25]:
tfidf = TfidfVectorizer()
# pca = PCA(n_components=200, random_state=42)
svd = TruncatedSVD(n_components=2, n_iter=75, random_state=42)
tfidf_full = tfidf.fit_transform(full_text)
# tfidf_train_dense = tfidf_train.todense()
# tfidf_train_pca = pca.fit_transform(tfidf_train_dense)
tfidf_full_svd = svd.fit_transform(tfidf_full)

In [26]:
#FOR FULL_DATA
X = tfidf_full_svd#full_data['Text']#.drop(['Author', 'encoded_author'], axis=1)
y = full_data['Author']#['cluster']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
len(X_train), len(X_test), len(X_val), len(y_train), len(y_test), len(y_val)

(3000, 1000, 1000, 3000, 1000, 1000)

In [137]:
lc = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = lc.fit(pd.DataFrame(X_train), pd.DataFrame(X_test), list(y_train), list(y_test))

100%|██████████| 29/29 [00:13<00:00,  2.21it/s]


In [139]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.13,0.13,None,0.13,0.97
ExtraTreesClassifier,0.13,0.13,None,0.13,0.62
LGBMClassifier,0.13,0.13,None,0.13,3.22
DecisionTreeClassifier,0.12,0.12,None,0.12,0.04
BaggingClassifier,0.12,0.12,None,0.12,0.17
ExtraTreeClassifier,0.12,0.12,None,0.12,0.02
QuadraticDiscriminantAnalysis,0.09,0.09,None,0.06,0.02
SVC,0.09,0.09,None,0.06,0.66
KNeighborsClassifier,0.09,0.08,None,0.08,0.04


In [248]:
vec = TfidfVectorizer()
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(vec, svd, clf)
pipe.fit(train_text, train_auth)
pipe.score(test_text, test_auth)

0.0728

In [ ]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(test_auth, y_pred):
        print("{:.3f} {}".format(prob, target))

doc = exp_text#test_text[4]
print_prediction(doc)

### LIME

In [117]:
classes = train_data['Author'].unique()
explainer = LimeTextExplainer(class_names=classes)
exp_text = test_text[4]
lime_exp = explainer.explain_instance(exp_text, pipe.predict_proba, top_labels=1)#pipe.predict_proba)#, num_features=(len(exp_text)//5))
# pipe.predict_proba(exp_text)

In [ ]:
lime_exp.show_in_notebook()
# test_auth[4]
pred = lime_exp.top_labels
pred

In [129]:
word_importances = lime_exp.as_list(pred[0])
word_importances

[('China', 0.10385065678995657),
 ('said', 0.035556795701384945),
 ('analysts', 0.021809420333518296),
 ('expansion', -0.01924900765830568),
 ('Monday', 0.01912818641042654),
 ('float', -0.01569811679007957),
 ('fund', -0.006403119697049611),
 ('network', -0.004723409506969597),
 ('plans', 0.004157887679415947),
 ('stock', 0.003607507104793779)]

## Word Embeddings

In [27]:
from gensim.models import KeyedVectors

# load the pre-trained FastText model
model_path = r"wiki-news-300d-1M-subword.vec\wiki-news-300d-1M-subword.vec"
vec_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

# check the size of the vocabulary and the vector size
print(f"Vocabulary size: {len(vec_model.index_to_key)}")
print(f"Vector size: {vec_model.vector_size}")
len(vec_model)

Vocabulary size: 999994
Vector size: 300


999994

In [28]:
def get_sentence_embedding(sentence):
    # Tokenize the sentence into individual words
    tokens = sentence.split()
    # Get the embedding for each word in the sentence
    embeddings = []
    for token in tokens:
        if token in vec_model:
            embeddings.append(vec_model[token])
    # Calculate the average embedding for the sentence
    if len(embeddings) > 0:
        sentence_embedding = np.mean(embeddings, axis=0)
    else:
        sentence_embedding = np.zeros((300,))
    return sentence_embedding

In [29]:
# Calculate the embeddings for each sentence
w2v_full = full_data['Text'].apply(get_sentence_embedding).tolist()

# Convert the embeddings list into a numpy array
w2v_full = np.array(w2v_full)

# Print the shape of the embeddings array
print(w2v_full.shape)


(5000, 300)


In [30]:
svd_w2v = TruncatedSVD(n_components=2, n_iter=75, random_state=42)
w2v_full = svd_w2v.fit_transform(w2v_full)
clf = LinearSVC()
w2v_pipe = make_pipeline(clf)


# Silhouette Score

In [44]:
#the higher the better
def sil_score(num_clusters, df):
    algort = KMeans(num_clusters).fit(df)
    print(silhouette_score(df,algort.labels_))

# Attribution Score

In [31]:
def get_attribution_score(word_attributions):
    total = 0
    for _, value in word_attributions:
        total += value
    return total

# Get Top Words

In [32]:
import spacy
import string
def get_topk(text, word_attributions):
    # recombined_words = get_full_words(text, word_attributions)
    top_words = []

    stop_words = set(stopwords.words('english'))
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(text) #the most recent sentence form; keep to revert back to after subbing a word
    doc_ents = [str(ent) for ent in doc.ents] #get the entities to skip over



    sorted_words = sorted(word_attributions, key=lambda x: x[1], reverse=True)
    positive_words = [t[0] for t in sorted_words if t[1] > 0]

    for word in positive_words: #loop through top k words
        if word in doc_ents or word in stop_words or any(punc in word for punc in string.punctuation): #if key is an entity, skip it
            # print("NER")
            continue
        else:
            top_words.append(word)
    
    return top_words

# Get Synonyms

## WordNet

In [33]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to get the top k synonyms for a word
def get_synonyms_wn(word, k=5):
    # Get the synsets for the word
    synsets = wordnet.synsets(word)
    # Get the lemmas for each synset and add them to a set
    synonyms = set()
    for synset in synsets:
        for lemma in synset.lemmas():
            # Lemmatize the lemma and add it to the set of synonyms
            lemma = lemmatizer.lemmatize(lemma.name())
            if lemma != word:
                synonyms.add(lemma)
    # Convert the set to a list and return the top k synonyms
    synonyms = [syn.replace("_", " ") for syn in synonyms]
    return list(synonyms)[:k]


def get_syn_dict_wn(words):
    # Dictionary to hold the synonyms
    synonyms_dict = {}

    # Get the synonyms for each word and add them to the dictionary
    for word in words:
        synonyms = get_synonyms_wn(word)
        synonyms_dict[word] = synonyms

    # Print the dictionary
    # print(synonyms_dict)
    return synonyms_dict

## FastText Model with Gensim

In [34]:
from gensim.models import KeyedVectors

# load the FastText model
# model = KeyedVectors.load_word2vec_format('path/to/fasttext.bin', binary=True)

def get_synonyms_kv(word, k=5):
    try:
        # get the k most similar words to the given word from the FastText model
        synonyms = vec_model.most_similar(word, topn=k)
        return [synonym[0] for synonym in synonyms]
    except KeyError:
        # handle the case where the word is not in the FastText model
        return []


def get_syn_dict_kv(words):
    # Dictionary to hold the synonyms
    synonyms_dict = {}

    # Get the synonyms for each word and add them to the dictionary
    for word in words:
        synonyms = get_synonyms_kv(word)
        synonyms_dict[word] = synonyms

    # Print the dictionary
    # print(synonyms_dict)
    return synonyms_dict


# Sub Words

In [36]:
#sandbox
import spacy
import re
# nlp = spacy.load("en_core_web_sm")

# NER = ['ORG', 'PERSON', 'GPE', 'DATE', 'TIME', 'PRODUCT']
# sent = "Today is a Tuesday , tomorrow will be Wednesday. We will be going to Mercury Finance Co. on Thursday!"
# topkwords = {"Tuesday": ["Thursday", "BLANK"], "be": ["is", "BLANK"], "Mercury Finance Co.": ["BLANK"]} #{topk: [synonyms]}

# orig_attrib_score = get_attribution_score(word_attributions)
# orig_label = true_auth
# new_label = ""
# curr_sent = sent #the most recent sentence form; keep to revert back to after subbing a word
# test_sent = "" #for testing out replacement words
# new_sent = "" #what is returned with the best subs


def sub_words(original_text,synonym_dict, original_label, original_attribution_score):
    # doc = nlp(original_text) #the most recent sentence form; keep to revert back to after subbing a word
    # doc_ents = [str(ent) for ent in doc.ents] #get the entities to skip over

    curr_text = original_text
    curr_text = re.sub(r'\s+', ' ', curr_text)
    
    test_text = "" #for testing out replacement words
    new_label = ""

    for key in synonym_dict: #loop through top k words
    #     if key in doc_ents: #if key is an entity, skip it
    #         # print("NER")
    #         continue
        if synonym_dict[key]:#make sure the list of synonyms for a key isn't empty
            for value in synonym_dict[key]: #loop through list of values for each top k word
                test_text = curr_text.replace(key, value) #replace keyword

                lime_exp = explainer.explain_instance(test_text, pipe.predict_proba, top_labels=1,num_features=word_count_func(test_text))
                pred = lime_exp.top_labels[0]

                word_attributions = lime_exp.as_list(pred)

                # word_attributions = cls_explainer(test_text) #reclassify with replacement
                # del word_attributions[0]
                
                new_attrib_score = get_attribution_score(word_attributions) #get new attribution score
                new_label = lime_exp.top_labels[0]
                # new_label = cls_explainer.predicted_class_name

                if new_label != original_label: #SUCCESS
                    print("SUCCESS!")
                    curr_text = test_text
                    return curr_text
                
                elif new_attrib_score < original_attribution_score:
                    curr_text = test_text #updating the current sentence with the new word that caused the attribution score to drop
                else: #reset sentence back to previous to try other values if the score doesnt drop or the label doesnt change
                    test_text = curr_text 
        else:#go to the next key
            continue
    print("Unable to change label.")
    return curr_text#, new_label

# Adversarial Generator

In [37]:
#loop through multiple samples
results = pd.DataFrame(columns=['Original Label', 'Original Attribution Score', 'Original Text', 'New Label', 'New Attribution Score', 'New text', 'Misclassified'])

def generate_adv(text):
    wc = word_count_func(text)
    lime_exp = explainer.explain_instance(text, pipe.predict_proba, top_labels=1,num_features=wc)
    original_label = lime_exp.top_labels[0]
    word_attributions = lime_exp.as_list(original_label)
    # word_attributions = cls_explainer(text)
    # del word_attributions[0]
    original_attribution_score = get_attribution_score(word_attributions)


    top_words = get_topk(text, word_attributions)
    
    syn_dict = get_syn_dict_wn(top_words)
    # syn_dict = get_syn_dict_kv(top_words)

    adv = sub_words(text, syn_dict, lime_exp.top_labels, original_attribution_score)
    wc = word_count_func(adv)
    # print(f"ADV: {adv}")
    lime_exp = explainer.explain_instance(adv, pipe.predict_proba, top_labels=1,num_features=wc)
    adv_label = lime_exp.top_labels[0]
    adv_attributions = lime_exp.as_list(adv_label)
    adv_attrib_score = get_attribution_score(adv_attributions) 
    if original_label != adv_label:
        misclassified = True
    else:
        misclassified = False

    return {'Original Label': original_label, 'Original Attribution Score': original_attribution_score,
             'Original Text': text, 'New Label': adv_label, 'New Attribution Score': adv_attrib_score, 
             'New text': adv, 'Misclassified': misclassified}

# TFIDF Clustering

In [38]:
repetitions = ['R{}'.format(i) for i in range(1,10)]

SIL_results = pd.DataFrame( index = range(2,25), 
                    columns= repetitions)


for n_cluster in SIL_results.index:
    for col in SIL_results.columns:
        algort = KMeans(n_clusters=n_cluster, random_state=42).fit(tfidf_full_svd)
        SIL_results.at[n_cluster,col] = silhouette_score(tfidf_full_svd,algort.labels_)
        
SIL_results['Mean'] = SIL_results[repetitions].mean(axis=1)

SIL_results['Var'] = SIL_results[repetitions].var(axis=1)
SIL_results.sort_values('Mean',ascending=False)

,R1,R2,R3,R4,R5,R6,R7,R8,R9,Mean,Var
2,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.00
3,0.38,0.38,0.38,0.38,0.38,0.38,0.38,0.38,0.38,0.38,0.00
5,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.00
8,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.00
11,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.34,0.00
10,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
7,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
9,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
4,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00
13,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.00


In [98]:
num_clusters = 5
km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
km.fit(tfidf_full_svd)
silhouette_avg = silhouette_score(tfidf_full_svd, km.labels_)
print('Silhouette Score:', silhouette_avg)
print(f"CH INDEX: {calinski_harabasz_score(tfidf_full_svd, km.labels_)}")
print(f"DB INDEX: {davies_bouldin_score(tfidf_full_svd, km.labels_)}")

Silhouette Score: 0.3435989444274543
CH INDEX: 4264.073737946438
DB INDEX: 0.9103115415975793


In [102]:
tfidf_df = pd.DataFrame(tfidf_full_svd)
tfidf_df['cluster'] = km.labels_
tfidf_df['Text'] = full_data['Text']
tfidf_df['Author'] = full_data['Author']
tfidf_df
# tfidf_df.to_csv('data\c50_tfidf_clustered.csv')

,0,1,cluster,Text,Author
0,0.16,0.04,1,the internet may be overflowing with new techn...,candidate00001
1,0.17,0.03,0,the us postal service announced wednesday a pl...,candidate00001
2,0.08,-0.00,1,elementary school students with access to the ...,candidate00001
3,0.15,0.05,1,an influential internet organisation has backe...,candidate00001
4,0.15,0.05,1,an influential internet organisation has backe...,candidate00001
...,...,...,...,...,...
4995,0.19,-0.08,4,britains big banks look set to raise profits b...,candidate00018
4996,0.21,0.03,0,after two years of hype and euphoria about the...,candidate00047
4997,0.08,-0.01,1,czech annual average consumer inflation eased ...,candidate00002
4998,0.18,-0.03,2,kellogg co whose profits for 1996 are under pr...,candidate00037


In [100]:
#FOR FULL_DATA
# X = tfidf_df.iloc[:,:2]#drop(tfidf_df.columns[:-2], axis=1)#full_data['Text']#.drop(['Author', 'encoded_author'], axis=1)
X = tfidf_df['Text']
y = tfidf_df['cluster']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
len(X_train), len(X_test), len(X_val), len(y_train), len(y_test), len(y_val)

(3000, 1000, 1000, 3000, 1000, 1000)

In [245]:
X_test =X_test.reset_index(drop=True)
y_test =y_test.reset_index(drop=True)

X_val =X_val.reset_index(drop=True)
y_val =y_val.reset_index(drop=True)

X_train =X_train.reset_index(drop=True)
y_train =y_train.reset_index(drop=True)

In [246]:
lc = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = lc.fit(pd.DataFrame(X_train), pd.DataFrame(X_test), list(y_train), list(y_test))

100%|██████████| 29/29 [00:05<00:00,  5.05it/s]


In [238]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.99,0.99,None,0.99,0.23
RandomForestClassifier,0.98,0.98,None,0.98,0.35
SVC,0.98,0.98,None,0.98,0.10
XGBClassifier,0.98,0.98,None,0.98,0.32
DecisionTreeClassifier,0.98,0.98,None,0.98,0.01
LGBMClassifier,0.98,0.97,None,0.98,0.33
LabelPropagation,0.97,0.97,None,0.97,0.22
LabelSpreading,0.97,0.97,None,0.97,0.31
BaggingClassifier,0.97,0.97,None,0.97,0.06


In [253]:
vec = TfidfVectorizer()#(min_df=3, stop_words='english', ngram_range=(1, 2))
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
clf = SVC(probability=True)#C=150, gamma=2e-2, probability=True)

pipe = make_pipeline(vec, svd, clf)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.901

In [254]:
X_test

0      australian life office national mutual holding...
1      fresh from a highcourt victory floridas antito...
2      calenergy inc upped the ante in its hostile pu...
3      united news amp media on tuesday secured victo...
4      leading chinese dissident liu xiaobo has been ...
                             ...                        
995    the story of brex minerals ltd and its fabulou...
996    compaq computer corp the worlds largest person...
997    liggett groups pact breaking away from other c...
998    australias westpac banking corp struggling lik...
999    some 120 eurotunnel sa plc employees are worki...
Name: Text, Length: 1000, dtype: object

## LIME

In [291]:
classes = tfidf_df['cluster'].unique()
explainer = LimeTextExplainer(class_names=classes)
exp_text = X_test[9]
actual_auth = y_test[9]
lime_exp = explainer.explain_instance(exp_text, pipe.predict_proba, top_labels=1, num_features=word_count_func(exp_text))#pipe.predict_proba)#, num_features=(len(exp_text)//5))
# pipe.predict_proba(exp_text)

In [358]:
# lime_exp.show_in_notebook()
pred = lime_exp.top_labels
# actual_auth, 
pred

[1]

In [362]:
lime_exp = explainer.explain_instance(exp_text, pipe.predict_proba, top_labels=1,num_features=word_count_func(exp_text))
pred = lime_exp.top_labels[0]

word_attributions = lime_exp.as_list(pred) #list of tuples
word_attributions


[('donor', 0.07305992431982994),
 ('and', -0.050646822323964544),
 ('to', -0.04739261648396057),
 ('blessing', 0.040956222865821686),
 ('in', -0.03781632012805696),
 ('with', -0.02821373660367552),
 ('loan', 0.024210585970232266),
 ('government', -0.02285278957981466),
 ('exporters', 0.017985306136978855),
 ('into', -0.016747851789042503),
 ('cocoa', 0.014500310367551242),
 ('come', 0.014039282435750864),
 ('line', 0.012731659459068057),
 ('conditions', 0.011423320017739082),
 ('coasts', 0.011400234520894802),
 ('auction', 0.011389560867105422),
 ('need', 0.010413243769381847),
 ('coffee', 0.010359718702811967),
 ('say', 0.009752780777860544),
 ('effect', 0.008865018662490254),
 ('system', 0.00781456380104643),
 ('revisions', -0.007790155967675714),
 ('have', -0.007111572724896516),
 ('ivory', 0.005185549114038121),
 ('sources', -0.004357614220335949),
 ('only', 0.0032038974379256747),
 ('been', 0.0013145931984821144),
 ('finalised', 0.0003745688759115644)]

In [ ]:
top_words = get_topk(exp_text, word_attributions)
# top_words
lime_exp.top_labels

In [ ]:
score = get_attribution_score(word_importances)

syns = get_syn_dict_wn(top_words)
syns

{'donor': ['giver', 'bestower', 'presenter', 'conferrer'],
 'blessing': ['grace', 'thanksgiving', 'hallow', 'sanctify', 'boon'],
 'need': ['motivation', 'indigence', 'want', 'pauperism', 'motive'],
 'loan': ['lend', 'loanword'],
 'cocoa': ['drinking chocolate', 'chocolate', 'hot chocolate'],
 'come': ['cum', 'arrive', 'amount', 'occur', 'descend'],
 'auction': ['auction bridge',
  'vendue',
  'auctioneer',
  'auction sale',
  'auction off'],
 'effect': ['impression', 'issue', 'result', 'outcome', 'event'],
 'exporters': ['exporter'],
 'coasts': ['glide', 'coast', 'sea-coast', 'seacoast', 'slide'],
 'conditions': ['status',
  'weather condition',
  'stipulation',
  'circumstance',
  'condition'],
 'coffee': ['java', 'burnt umber', 'chocolate', 'deep brown', 'coffee berry'],
 'ivory': ['bone', 'tusk', 'pearl', 'off-white'],
 'say': ['aver', 'allege', 'articulate', 'read', 'sound out'],
 'revisions': ['revise', 'rescript', 'rewrite', 'revision', 'alteration'],
 'system': ['organization',


In [355]:
res = sub_words(exp_text, syns, pred, score)
res

KeyError: 1

In [340]:
word_count_func(X_test[0])

34

In [373]:

for text in X_test[:5]:
    row = generate_adv(text)
    # print(row)
    results.loc[len(results)] = row
    # results.append(row, ignore_index=True)
results


SUCCESS!
ADV: Aussie life office national mutual holdings ltd is bear to make a firm historic debut on the Aussie and new zealand bourses on tuesday after its institutional offer closed six times oversubscribed last week
Unable to change label.
ADV: fresh from a highcourt triumph floridas antitobacco activist are fresh for a legislative assault on the innovative legal philosophy allow the state government to Eugene Sue cigarette makers
SUCCESS!
ADV: calenergy Iraqi National Congress upped the ante in its hostile pursuit of british power company northern electric plc friday raising its offer to 782 one thousand thousand pounds 129 billion and setting a dec 20 deadline
SUCCESS!
ADV: united news amp media on tuesday secured victory in a battle for control of international exhibitions group blenheim with an agreed offer valuing the company at 5925 one thousand thousand pounds 935 one thousand thousand
SUCCESS!
ADV: leading Taiwanese dissident liu xiaobo has been ordered to serve three year

,Original Label,Original Attribution Score,Original Text,New Label,New Attribution Score,New text,Misclassified
0,2,1.10,australian life office national mutual holding...,0,0.53,Aussie life office national mutual holdings lt...,True
1,1,0.04,fresh from a highcourt victory floridas antito...,1,0.04,fresh from a highcourt triumph floridas antito...,False
2,4,1.22,calenergy inc upped the ante in its hostile pu...,2,0.69,calenergy Iraqi National Congress upped the an...,True
3,4,1.33,united news amp media on tuesday secured victo...,2,0.44,united news amp media on tuesday secured victo...,True
4,3,1.03,leading chinese dissident liu xiaobo has been ...,1,-0.08,leading Taiwanese dissident liu xiaobo has bee...,True


# W2V Clustering

In [41]:
# repetitions = ['R{}'.format(i) for i in range(1,10)]

SIL_results = pd.DataFrame( index = range(2,25), 
                    columns= repetitions)


for n_cluster in SIL_results.index:
    for col in SIL_results.columns:
        algort = KMeans(n_clusters=n_cluster, repetitions = ['R{}'.format(i) for i in range(1,10)]

SIL_results = pd.DataFrame( index = range(2,25), 
                    columns= repetitions)


for n_cluster in SIL_results.index:
    for col in SIL_results.columns:
        algort = KMeans(n_clusters=n_cluster, random_state=42).fit(tfidf_full_svd)
        SIL_results.at[n_cluster,col] = silhouette_score(tfidf_full_svd,algort.labels_)
        
SIL_results['Mean'] = SIL_results[repetitions].mean(axis=1)

SIL_results['Var'] = SIL_results[repetitions].var(axis=1)
SIL_results.sort_values('Mean',ascending=False)).fit(w2v_full)
        SIL_results.at[n_cluster,col] = silhouette_score(w2v_full,algort.labels_)
        
SIL_results['Mean'] = SIL_results[repetitions].mean(axis=1)

SIL_results['Var'] = SIL_results[repetitions].var(axis=1)
SIL_results.sort_values('Mean',ascending=False)

,R1,R2,R3,R4,R5,R6,R7,R8,R9,Mean,Var
3,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.00
2,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.43,0.00
6,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.42,0.00
7,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.00
5,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.00
4,0.40,0.40,0.39,0.40,0.40,0.40,0.39,0.40,0.40,0.40,0.00
8,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.00
11,0.38,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.00
12,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.00
10,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.00


In [103]:
num_clusters = 6
km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
km.fit(w2v_full)
silhouette_avg = silhouette_score(w2v_full, km.labels_)
print('Silhouette Score:', silhouette_avg)
print(f"CH INDEX: {calinski_harabasz_score(w2v_full, km.labels_)}")
print(f"DB INDEX: {davies_bouldin_score(w2v_full, km.labels_)}")

Silhouette Score: 0.41612533
CH INDEX: 5402.987022039822
DB INDEX: 0.7978002245266111


In [104]:
km.labels_

array([4, 0, 3, ..., 1, 1, 3])

In [107]:
# full_data['cluster'] = km.labels_
# full_data.reset_index(drop=True)
# full_data
w2v_df = pd.DataFrame(w2v_full)
w2v_df['cluster'] = km.labels_
w2v_df['Text'] = full_data['Text']
w2v_df['Author'] = full_data['Author']
w2v_df#['cluster'].value_counts()
# w2v_df.to_csv('data\c50_w2v_clustered.csv')

,0,1,cluster,Text,Author
0,0.37,-0.07,4,the internet may be overflowing with new techn...,candidate00001
1,0.34,0.01,0,the us postal service announced wednesday a pl...,candidate00001
2,0.28,0.01,3,elementary school students with access to the ...,candidate00001
3,0.33,0.02,0,an influential internet organisation has backe...,candidate00001
4,0.33,0.02,0,an influential internet organisation has backe...,candidate00001
...,...,...,...,...,...
4995,0.33,-0.04,4,britains big banks look set to raise profits b...,candidate00018
4996,0.35,-0.01,0,after two years of hype and euphoria about the...,candidate00047
4997,0.23,-0.05,1,czech annual average consumer inflation eased ...,candidate00002
4998,0.29,-0.06,1,kellogg co whose profits for 1996 are under pr...,candidate00037


In [90]:
# counts = full_data['cluster'].value_counts()
# counts
tfidf_df['cluster'].value_counts()

cluster
1    1361
2    1211
0    1090
4     685
3     653
Name: count, dtype: int64

In [54]:
#FOR FULL_DATA
X = w2v_df.drop(w2v_df.columns[-1], axis=1)#full_data['Text']#.drop(['Author', 'encoded_author'], axis=1)
y = w2v_df['cluster']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
len(X_train), len(X_test), len(X_val), len(y_train), len(y_test), len(y_val)

(3000, 1000, 1000, 3000, 1000, 1000)

In [55]:
lc = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = lc.fit(pd.DataFrame(X_train), pd.DataFrame(X_test), list(y_train), list(y_test))

100%|██████████| 29/29 [00:00<00:00, 3614.92it/s]


In [56]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


In [158]:
clf = LogisticRegression()
pipe = make_pipeline(clf)
pipe.fit(X_train, y_train)
test_pred = pipe.predict(X_test)

In [236]:
from sklearn.metrics import classification_report


print(classification_report(y_test, test_pred,  target_names=['0', '1', '2', '3', '4', '5']))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88       212
           1       0.97      0.93      0.95       178
           2       0.00      0.00      0.00        51
           3       0.87      0.86      0.87       185
           4       0.94      1.00      0.97       224
           5       0.67      0.66      0.66       150

    accuracy                           0.86      1000
   macro avg       0.71      0.74      0.72      1000
weighted avg       0.81      0.86      0.83      1000



# Compare

In [4]:
w2v_cluster = pd.read_csv(r'data\c50_w2v_clustered.csv')
tfidf_cluster = pd.read_csv(r'data\c50_tfidf_clustered.csv')

In [91]:
tfidf_df['w2v'] = w2v_df['cluster']
tfidf_df

,0,1,cluster,Text,Author,w2v
0,0.16,0.04,1,the internet may be overflowing with new techn...,candidate00001,3
1,0.17,0.03,0,the us postal service announced wednesday a pl...,candidate00001,4
2,0.08,-0.00,1,elementary school students with access to the ...,candidate00001,1
3,0.15,0.05,1,an influential internet organisation has backe...,candidate00001,4
4,0.15,0.05,1,an influential internet organisation has backe...,candidate00001,4
...,...,...,...,...,...,...
4995,0.19,-0.08,4,britains big banks look set to raise profits b...,candidate00018,3
4996,0.21,0.03,0,after two years of hype and euphoria about the...,candidate00047,4
4997,0.08,-0.01,1,czech annual average consumer inflation eased ...,candidate00002,0
4998,0.18,-0.03,2,kellogg co whose profits for 1996 are under pr...,candidate00037,0


In [93]:
new_df = tfidf_df[tfidf_df['cluster'] != tfidf_df['w2v']]
new_df.reset_index(drop=True)

,0,1,cluster,Text,Author,w2v
0,0.16,0.04,1,the internet may be overflowing with new techn...,candidate00001,3
1,0.17,0.03,0,the us postal service announced wednesday a pl...,candidate00001,4
2,0.15,0.05,1,an influential internet organisation has backe...,candidate00001,4
3,0.15,0.05,1,an influential internet organisation has backe...,candidate00001,4
4,0.13,-0.00,1,when a company in california sells a book to a...,candidate00001,2
...,...,...,...,...,...,...
3971,0.19,-0.08,4,britains big banks look set to raise profits b...,candidate00018,3
3972,0.21,0.03,0,after two years of hype and euphoria about the...,candidate00047,4
3973,0.08,-0.01,1,czech annual average consumer inflation eased ...,candidate00002,0
3974,0.18,-0.03,2,kellogg co whose profits for 1996 are under pr...,candidate00037,0


In [81]:
import pandas as pd

df1 = pd.DataFrame({'col1': [0, 1, 2, 3], 'col2': ['a', 'b', 'c', 'd']})
df2 = pd.DataFrame({'col1': [0, 3, 4, 0], 'col3': ['x', 'y', 'z', 'w']})

merged_df = pd.merge(df1, df2, on='col1')
count = len(merged_df)
count

3